In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\prajw\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [2]:
url = 'https://in.indeed.com/'
driver.get(url)

job_input_box = driver.find_element_by_xpath('//*[@id="text-input-what"]')
job_input_box.send_keys('Data Scientist') # Job role

location_input_box = driver.find_element_by_xpath('//*[@id="text-input-where"]')
location_input_box.send_keys('Bangalore') #Job Location

location_input_box.send_keys(Keys.ENTER)

In [3]:
df = pd.DataFrame(columns=['Job Title', 'Company', 'Location',
                           'Salary', 'Requirements', 'Date Posted', 'Remote', 'Link'])

def scrape_jobs_from_page():
    for job in jobs:
        job_title = job.find('a', class_='jobtitle turnstileLink').text.strip()
        company = job.find('span', class_='company').text.strip()

        # some job locations are in div tag while others are in span but both have the same class,so enclose both tags in a list
        job_location = job.find(['span', 'div'], class_='location accessible-contrast-color-location').text

        # store salary if specified else store 'N/A'
        if job.find('span', class_='salaryText'):
            salary = job.find('span', class_='salaryText').text.strip()
        else:
            salary = 'N/A'

        job_requirements = job.find('div', class_='summary').text.strip()
        date_posted = job.find('span', class_='date').text.strip()

        # store if job is remote if specified else store ' - '
        if job.find('span', class_='remote'):
            remote = job.find('span', class_='remote').text.strip()
        else:
            remote = ' - '

        job_link = 'https://in.indeed.com' + job.find('a').get('href')

        index = len(df)
        df.loc[index] = [job_title, company, job_location, salary,
                         job_requirements, date_posted, remote, job_link]


while True:
    soup = BeautifulSoup(driver.page_source, parser='lxml')
    jobs = soup.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result clickcard')
    scrape_jobs_from_page()
    try:
        next_page_link = 'https://in.indeed.com' + soup.find('a', {'aria-label':'Next'}).get('href')
    except:
        print('Last page reached')
        break
    driver.get(next_page_link)

Last page reached


In [4]:
len(df)

495

In [7]:
df.head()

,Job Title,Company,Location,Salary,Requirements,Date Posted,Remote,Link
0,Data Scientist (NLP) for a Product Based Edute...,Shaw Academy,"Bengaluru, Karnataka",N/A,Prior experience in working with databases and...,30+ days ago,Temporarily remote,https://in.indeed.com/cmp/Shaw-Academy
1,Data Scientist,Pion Global Pvt Ltd,"Bengaluru, Karnataka",N/A,"Machine learning, data science, deep learning....",4 days ago,Temporarily remote,https://in.indeed.com/company/Growth-gamut/job...
2,Data Scientist,AceNet,"Bengaluru, Karnataka",N/A,Joining : Candidates who can join us immediate...,6 days ago,-,https://in.indeed.com/rc/clk?jk=405169173c513d...
3,Sr. Data Scientist,Qwickbit technologies pvt ltd,"Bengaluru, Karnataka","₹12,00,000 - ₹16,00,000 a year",\* Present information using data visualizatio...,18 days ago,Temporarily remote,https://in.indeed.com/company/Qwickbit-technol...
4,Data Scientist DA4AD,Mercedes-Benz Research and Development India P...,"Bengaluru, Karnataka",N/A,Experience with data visualisation tools like ...,30+ days ago,-,https://in.indeed.com/rc/clk?jk=60d4983b398b7f...


In [8]:
# found 495 'data scientist' jobs in 'Bangalore'
# now lets save all data into a csv file
df.to_csv('Data Scientist jobs in Bangalore.csv',index=False)